In [1]:
import os
from dotenv import load_dotenv

from qdrant_client import QdrantClient
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
import openai

e:\Ananconda\envs\lc_env\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
embedding_model = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

C:\Users\HP\AppData\Local\Temp\ipykernel_17480\2048681798.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
e:\Ananconda\envs\lc_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
file_path = r'C:\Users\HP\Desktop\langchain-crash-course\Practicing\Books\The Art of Being ALONE Solitude Is My HOME, Loneliness Was My Cage.pdf'
loader = PyPDFLoader(file_path)
doc = loader.load()

In [4]:
len(doc)

99

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=20)
chunks = splitter.split_documents(doc)

In [6]:
document = []
for chunk in chunks:
    meta_data = {'source':"AI learning", "metadata": chunk.metadata}
    doc_obj = Document(page_content=chunk.page_content, metadata=meta_data)
    document.append(doc_obj)

In [7]:
len(document)

304

In [8]:
document[8]

Document(metadata={'source': 'AI learning', 'metadata': {'source': 'C:\\Users\\HP\\Desktop\\langchain-crash-course\\Practicing\\Books\\The Art of Being ALONE Solitude Is My HOME, Loneliness Was My Cage.pdf', 'page': 6}}, page_content='That’s what you need, right? You don’t care how many\npeople in the world are lonely . You care about how you can\ndeal with your loneliness.\nWell, that’s the ﬁrst problem. Why is loneliness\nconsider ed such a taboo in our society? Why do you hate\nthe idea of being with yourself so much that ‘the\ntime you spend with yourself is now considered as\nloneliness’  and there should be some instant hacks and\ntechniques to deal with it? How can the idea of being with\nyourself mak e you feel scar ed?\nHonestly, I have no authority to ask these questions. I')

## Qdrant VectorStore

In [9]:
load_dotenv()

True

### Logic for getting the exect number of values we want 

In [12]:
main_lis= []
last_index= 0
for i in range(len(document)):
    if i%100 == 0 and i != 0:
        sub_lis = [i-100, i-1]
        last_index = i
        main_lis.append(sub_lis)
    elif i == len(document)-1:
        sub_lis = [last_index, i]
        main_lis.append(sub_lis)

main_lis

[[0, 99], [100, 199], [200, 299], [300, 303]]

In [14]:
for i in main_lis:    
    db = QdrantVectorStore.from_documents(documents=document[i[0]:i[1]],
                                          embedding=embedding_model,
                                          url = os.getenv("QDRANT_VS_URL"),
                                          api_key=os.getenv("QDRANT_VS_API_KEY"),
                                          collection_name="Qdrant_collection",
                                          timeout=200)

In [94]:
# Once you run this don't run it again, as it will again make the embeddings and
# thus ur search of ur data will take more time 

# if the write operation timeout try to make sure the number of chunks minimum

# qdrant = QdrantVectorStore.from_documents(documents=document[100:],
#                                           embedding=embedding_model,
#                                           url = os.getenv("QDRANT_VS_URL"),
#                                           api_key=os.getenv("QDRANT_VS_API_KEY"),
#                                           collection_name="Qdrant_collection")

In [16]:
query = "How i can belong to myself"

result = db.similarity_search(query=query, k=3)

In [17]:
result[1].metadata

{'source': 'AI learning',
 'metadata': {'source': 'C:\\Users\\HP\\Desktop\\langchain-crash-course\\Practicing\\Books\\The Art of Being ALONE Solitude Is My HOME, Loneliness Was My Cage.pdf',
  'page': 58},
 '_id': '0c01073f-0502-40e5-bb28-4c3c16508629',
 '_collection_name': 'Qdrant_collection'}

In [18]:
result[2].page_content

'As I said earlier, it’s liberating to be with yoursel f. You are'

In [19]:
query = "How i can belong to myself"

result = db.max_marginal_relevance_search(query=query, k=3)

In [20]:
for specfic_result in result:  
    print('Metadata:\n',specfic_result.metadata)
    print('\npage content:\n',specfic_result.page_content)

Metadata:
 {'source': 'AI learning', 'metadata': {'source': 'C:\\Users\\HP\\Desktop\\langchain-crash-course\\Practicing\\Books\\The Art of Being ALONE Solitude Is My HOME, Loneliness Was My Cage.pdf', 'page': 58}, '_id': 'e43599a1-edde-4015-a385-385543378160', '_collection_name': 'Qdrant_collection'}

page content:
 own life and build a life that makes you feel proud of
yourself, let’s learn to use your alone time to turn into your
growth time. Shall we?
Metadata:
 {'source': 'AI learning', 'metadata': {'source': 'C:\\Users\\HP\\Desktop\\langchain-crash-course\\Practicing\\Books\\The Art of Being ALONE Solitude Is My HOME, Loneliness Was My Cage.pdf', 'page': 35}, '_id': '6b1bc43d-519d-4f9a-9069-c30b607110e7', '_collection_name': 'Qdrant_collection'}

page content:
 Be undeﬁnable. Let them wonder who you are while you
create a personality for yourself that makes you fall in love
with yourself every day .
Metadata:
 {'source': 'AI learning', 'metadata': {'source': 'C:\\Users\\HP\\Deskto

### You can also define your Qdrant model in the following way:

In [21]:
from qdrant_client.http import models

client = QdrantClient(
    url=os.getenv("QDRANT_VS_URL"), api_key=os.getenv("QDRANT_VS_API_KEY")
)
collection_config = models.VectorParams(
    size=3072,
    distance=models.Distance.COSINE
)
client.create_collection(
    collection_name="collection_2",
    vectors_config=collection_config
)

# this will make your collection in the qdrant collection


True

# PineCone

In [27]:
from langchain_pinecone import PineconeVectorStore

In [34]:
load_dotenv()

True

In [36]:
pc_vector_store = PineconeVectorStore.from_documents(
    document,
    embedding_model,
    # api_key=os.getenv('PINECONE_API_KEY'),
    index_name= "ai-practice"
)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '878787a60f1a580b7ee05089d9f3eb00', 'Date': 'Fri, 27 Sep 2024 04:44:23 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key
